# Pyvista demo in ThDSubduction

In this notebook, I showcase the usage of the Pyvista package in processing vtu dataset and generate easy-to-plot subset.

In [ ]:
import os, sys

import numpy as np
from pathlib import Path

root_path = os.path.join(Path().resolve().parent.parent.parent)

if os.path.abspath(root_path) not in sys.path:
    sys.path.append(os.path.abspath(root_path))

# Settings

- case directory
- pvtu step number to plot, the file name is: 
    "solution-%05d.pvtu" % pvtu_step

In [ ]:
# case name and directories
local_ThDSubduction_dir = "/mnt/lochy/ASPECT_DATA/ThDSubduction"
case_name = "chunk_geometry1/eba3d_width80_bw8000_sw2000_yd500.0_AR4"
pvtu_step = 105 # 2 or 15

if case_name is not None:
    local_dir = os.path.join(local_ThDSubduction_dir, case_name)
    assert(os.path.isdir(local_dir))

pyvista_outdir = os.path.join(local_dir, "pyvista_outputs")
if not os.path.isdir(pyvista_outdir):
    os.mkdir(pyvista_outdir)

# Processing

- Make a domain boundary as well as the mark points.
- Read the vtu file and process a list of outputs, include domain surface and domain center slices, iso-volume object of diffferent compositions as well as the points on the slab surface and plate edge surface.
- Generate a connected shape of the subductiing slab.

In [ ]:
from hamageolib.research.haoyuan_3d_subduction.post_process import PYVISTA_PROCESS_THD

In [ ]:
# initiate the object
PprocessThD = PYVISTA_PROCESS_THD(pyvista_outdir=pyvista_outdir)

In [ ]:
# make domain boundary
p_marker_coordinates = {"r": 6371e3 - np.arange(0, 6000e3, 1000e3), "lon": np.arange(0, 90, 10)*np.pi/180.0, "lat": np.arange(0, 50.0, 10.0)*np.pi/180.0}
PprocessThD.make_boundary(marker_coordinates=p_marker_coordinates)

In [ ]:
# read vtu file
pvtu_filepath = os.path.join(local_dir, "output", "solution", "solution-%05d.pvtu" % pvtu_step)
PprocessThD.read(pvtu_step, pvtu_filepath)
# slice at center
PprocessThD.slice_center()
# slice at surface
PprocessThD.slice_surface()
# slice at depth
PprocessThD.slice_at_depth(200e3)
# extract sp_upper composition beyond a threshold
PprocessThD.extract_iso_volume_upper(0.8)
# extract sp_lower composition beyond a threshold
PprocessThD.extract_iso_volume_lower(0.8)
# extract plate_edge composition beyond a threshold
PprocessThD.extract_plate_edge(0.8)
# extract slab surface
PprocessThD.extract_slab_surface()
# extract slab edge
PprocessThD.extract_plate_edge_surface()
# filter the slab lower points
PprocessThD.filter_slab_lower_points()

# Visualization

In Paraview, open the generated files.

- Sp_lower_above_0.8_filtered_pe

    Plot by radius (depth 1000 to depth 0);
    Color - imola (opacity 0.05)

- slice_center

    Add Glyph:
    Maximum number of Sample - 1000;
	Scale Factor - 5e6;
	Glyph mode - …(Surface sampling);
	Line Width - 2;
    Color - solid color (purple, row 3, column 2);

- slice_depth_200km

    Add another Glyph.